In [1]:
import import_ipynb
import matplotlib.pyplot as plt
from FULL_DATA import final_df
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import regexp_tokenize
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

importing Jupyter notebook from FULL_DATA.ipynb
importing Jupyter notebook from Create_Labels.ipynb
(6380, 4)
4377
(4400, 7)
(4377, 7)
(4437, 7)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\grays\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\grays\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
final_df['SENTIMENT'] = [-1 if (x > 1 and x<3.4999) else 0 if (x > 3.5 and x<4.49999) else 1  for x in final_df['ratings']]
final_df['SENTIMENT'].value_counts()

 0    2367
-1    1144
 1     866
Name: SENTIMENT, dtype: int64

In [3]:
### Multi-class text classification
default_pattern =  r"""(?x)                  
                        (?:[A-Z]\.)+          
                        |\$?\d+(?:\.\d+)?%?    
                        |\w+(?:[-']\w+)*      
                        |\.\.\.               
                        |(?:[.,;"'?():-_`])    
                    """
def tokenize(text, pattern = default_pattern):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return regexp_tokenize(text, pattern)
final_df['TOKENIZED_TRANSCRIPT'] = final_df.apply(lambda row: tokenize(row['TRANSCRIPTS']), axis=1)
final_df['TOKENIZED_TRANSCRIPT'] = final_df['TOKENIZED_TRANSCRIPT'].apply(lambda x: ' '.join(x))

In [5]:
cv = CountVectorizer(ngram_range = (1,1),max_df = .9,min_df=.1)
text_counts = cv.fit_transform(final_df['TOKENIZED_TRANSCRIPT'])


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    text_counts, final_df['SENTIMENT'], test_size=0.3, random_state=1)

In [9]:
clf = MultinomialNB().fit(X_train, y_train)
clf = MultinomialNB().fit(X_train, y_train)
print("Test MultinomialNB Accuracy:",metrics.accuracy_score(y_test, clf.predict(X_test)))
print("MultinomialNB F1:",metrics.f1_score(y_test, clf.predict(X_test),average = 'macro'))

Test MultinomialNB Accuracy: 0.4726027397260274
MultinomialNB F1: 0.4678522621487973


In [11]:
clf = LogisticRegression(random_state=1, multi_class= 'multinomial',solver = 'lbfgs',max_iter=200).fit(X_train, y_train)
print("Logistic Accuracy:",metrics.accuracy_score(y_test, clf.predict(X_test)))
print("Logistic F1:",metrics.f1_score(y_test, clf.predict(X_test),average = 'macro'))

Logistic Accuracy: 0.6293759512937596
Logistic F1: 0.5794550298189284


C:\Users\grays\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [13]:
clf = RandomForestClassifier(max_depth=5, random_state=1,n_estimators=1000)
clf.fit(X_train, y_train)
print("Test RF Accuracy:",metrics.accuracy_score(y_test, clf.predict(X_test)))
print("RF F1:",metrics.f1_score(y_test, clf.predict(X_test),average = 'macro'))

Test RF Accuracy: 0.558599695585997
RF F1: 0.2389322916666667


C:\Users\grays\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
